In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori
from scoop.msg import stone_pose_ver
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
from geometry_msgs.msg import WrenchStamped
import time
rospy.init_node('scoop', anonymous=True) #initialize the node



In [2]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1612508317.908788]: Client test: Starting sending goals


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [4]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass

In [5]:
def go_to_home():
    
    Hong_joint0 = radians(101.94)
    Hong_joint1 = radians(-83.72)
    Hong_joint2 = radians(-130.33)
    Hong_joint3 = radians(-55.98)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.02)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    while True:
        rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
        camPstone = np.array([pose_x, pose_y, pose_z, 1])
        eeTcam = m3d.Transform()
        eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
        eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
        eeTcam.orient.rotate_xb(eeTcam_e[0])
        eeTcam.orient.rotate_yb(eeTcam_e[1])
        eeTcam.orient.rotate_zb(eeTcam_e[2])
        tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
        eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
        tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
        worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
        worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
        if worldTstone[2]<0:
            continue
        else:
            return [worldTstone[0]+0.02+0.0025+0.003, worldTstone[1]+0.02148-0.00353-0.0006, max(worldTstone[2], 0.037)]

In [7]:
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, Go_stone_dir_sorted, yaw_pitch_normal_set
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z
        yaw = msg.yaw
        pitch = msg.pitch
        normal = msg.normal
        dir_depth_inner = msg.dir_depth_inner
        dir_depth_outer = msg.dir_depth_outer
        dir_depth_outer_minus_inner = []
        for i in range(len(dir_depth_inner)):
            dir_depth_outer_minus_inner.append(dir_depth_outer[i]-dir_depth_inner[i])
        dir_depth_outer_minus_inner_argsort = np.argsort(-np.array(dir_depth_outer_minus_inner)).tolist()
        dir_depth_outer_minus_inner_argsort_copy = dir_depth_outer_minus_inner_argsort[:]
        for index in dir_depth_outer_minus_inner_argsort:
            if dir_depth_outer[index]-dir_depth_inner[index]<0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
            elif dir_depth_outer[index]==0 or dir_depth_inner[index]==0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
        dir_depth_outer_minus_inner_argsort=dir_depth_outer_minus_inner_argsort_copy[:]
        hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        Go_stone_dir_sorted.append(dir_depth_outer_minus_inner_argsort)
        yaw_pitch_normal_set.append([yaw,pitch,normal])
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori, hori_stone_pose_callback)

In [8]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, gripper_direction_set, gripper_aperture_set, json_error
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        gripper_direction_set = []
        gripper_aperture_set = []
    else:
        try:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z 
            box = json.loads(msg.box)
            ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            #print box
            length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
            length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
            if length1 > length2:
                try:
                    if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                    gripper_direction_set.append(temp_angle)
                except:
                    gripper_direction_set.append(0)
                gripper_aperture_set.append(length2+0.006)
            else:
                try:
                    if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                    gripper_direction_set.append(temp_angle)
                except:
                    gripper_direction_set.append(0)
                gripper_aperture_set.append(length1+0.006)       
        except:
            json_error = True
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver, ver_stone_pose_callback)

In [9]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [10]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [11]:
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_finger_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    return traslating_wst_world, extension_distance

def pos_flength_adjust_thumb_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    traslating_wst_world[0]=traslating_wst_world[0]-((2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)-(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r))/sin(theta)
    return traslating_wst_world, extension_distance

In [12]:
from __future__ import division
import pybullet as p
import pybullet_data
import numpy as np
import time
import argparse
import pybullet_utils
from collision_utils import get_collision_fn
from scipy.spatial.transform import Rotation as R
from math import *

THUMB_JOINT_INDICES = [0]

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--birrt', action='store_true', default=False)
    parser.add_argument('--smoothing', action='store_true', default=False)
    args = parser.parse_args(args=[])
    return args

def set_joint_positions(body, joints, values):
    assert len(joints) == len(values)
    for joint, value in zip(joints, values):
        p.resetJointState(body, joint, value)
args = get_args()
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setGravity(0, 0, -9.8)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, False)
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, True)
p.resetDebugVisualizerCamera(cameraDistance=1.400, cameraYaw=58.000, cameraPitch=-42.200, cameraTargetPosition=(0.0, 0.0, 0.0))
CLIENT = 0
# load objects
finger = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Finger-2.9-urdf.urdf', basePosition=[0, 0, 0], baseOrientation=[0,0,1,0], useFixedBase=False)
thumb = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Assem-1.9-urdf.urdf', basePosition=[0, 0, 0], baseOrientation=[0,0,1,0],useFixedBase=False)
bowl_circular = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Bowl-circular-urdf.urdf', basePosition=[0.05, 0.69, 0.03], baseOrientation=[0,0,0,1], useFixedBase=True)
obstacles = [bowl_circular]

def skim_grasp_collision_check(Go_stone_pose, alpha, beta, psi=30*pi/180): # radius
    #time0=time.time()
    start_conf_thumb = [-0.0133]
    fingerbaseOrientation_raw = R.from_dcm([[0, 1, 0], [1, 0, 0], [0, 0, -1]]) * R.from_euler('z', alpha, degrees=False) * R.from_euler('y', beta, degrees=False)
    fingerbaseOrientation = fingerbaseOrientation_raw.as_quat().tolist()
    ThumbTFinger_normal = fingerbaseOrientation_raw.as_dcm()[:,0].tolist()
    print ThumbTFinger_normal
    fingerbasePosition = [Go_stone_pose[0]+0.006*ThumbTFinger_normal[0], Go_stone_pose[1]+0.006*ThumbTFinger_normal[1], Go_stone_pose[2]+0.006*ThumbTFinger_normal[2]]
    p.resetBasePositionAndOrientation(finger, fingerbasePosition, fingerbaseOrientation)
    finger_base_position = list(p.getLinkState(finger, 0)[0])
    thumbbaseOrientation = fingerbaseOrientation
    thumbbasePosition = [finger_base_position[0]-0.045*ThumbTFinger_normal[0], finger_base_position[1]-0.045*ThumbTFinger_normal[1], finger_base_position[2]-0.045*ThumbTFinger_normal[2]]
    set_joint_positions(thumb, THUMB_JOINT_INDICES, start_conf_thumb)
    p.resetBasePositionAndOrientation(thumb, thumbbasePosition, thumbbaseOrientation)
    collision_fn0 = len(p.getClosestPoints(bodyA=finger, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    collision_fn1 = len(p.getClosestPoints(bodyA=thumb, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    fingerbaseOrientation = (fingerbaseOrientation_raw * R.from_euler('y', psi, degrees=False)).as_quat().tolist()
    p.resetBasePositionAndOrientation(finger, fingerbasePosition, fingerbaseOrientation)
    finger_base_position = list(p.getLinkState(finger, 0)[0])
    thumbbaseOrientation = fingerbaseOrientation
    ThumbTFinger_normal = R.from_quat(fingerbaseOrientation).as_dcm()[:,0].tolist()
    thumbbasePosition = [finger_base_position[0]-0.045*ThumbTFinger_normal[0], finger_base_position[1]-0.045*ThumbTFinger_normal[1], finger_base_position[2]-0.045*ThumbTFinger_normal[2]]  
    p.resetBasePositionAndOrientation(thumb, thumbbasePosition, thumbbaseOrientation)
    collision_fn2 = len(p.getClosestPoints(bodyA=finger, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    collision_fn3 = len(p.getClosestPoints(bodyA=thumb, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    print p.getClosestPoints(bodyA=thumb, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)
    result = collision_fn0 or collision_fn1 or collision_fn2 or collision_fn3
    return result 

#skim_grasp_collision_check([0.021606433562704887, 0.7103900159992532, 0.03481510996124948], 1.61191721201, -0.419393650179, 0.698131700798)
#skim_grasp_collision_check([0.08080312757252475, 0.6952931065435054, 0.03300760952182907], 0, 0, 0)

In [13]:
def skim_grasp_execute(Go_index):
    print 'Go_index', index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    ####################### change tcp pose ############################
    rob.set_tcp((0.014, 0, 0.342, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    go_to_safe_place()    
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    #print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    #print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    #print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.342,-0.014,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    #print tcpTee

    n_normalize = normal / np.linalg.norm(normal)
    #print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    #print "tcpTnormal", tcpTnormal
    alpha0 = atan2(tcpTnormal[1], tcpTnormal[0]) # is yaw
    if str(alpha0) == 'nan':
        return False
    beta = -(pi/2+atan2(tcpTnormal[2], sqrt(tcpTnormal[0]**2+tcpTnormal[1]**2))) #tune psi
    if str(beta) == 'nan':
        return False
    if abs(beta)>(35*pi/180.0):
        return False
    if abs(beta*180/pi)>20:
        psi = 40*pi/180
    else:
        psi = 30*pi/180
    alpha_set = [alpha0]
    if abs(beta)<(5*pi/180):
        alpha_set = [-pi*3/4, -pi/2, -pi/4, 0, pi/4, pi/2, pi*3/4, pi]
    for alpha in alpha_set:
    #print 'collision_check_data', hori_pose_Go_stone_set[Go_index], alpha, beta, psi
        if skim_grasp_collision_check(hori_pose_Go_stone_set[Go_index], alpha, beta, psi)==True:
            print 'collision happen'
            continue
        Robotiq.goto(robotiq_client, pos=2.5*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1080')
        signal.setitimer(signal.ITIMER_REAL, 0)
        rob.movel_tool((tcpTstone[0,0]+0.003, tcpTstone[1,0]-0.002, 0, 0, 0, 0), acc=0.8, vel=2, wait=True)
        rob.translate_tool((0, 0, tcpTstone[2,0]-0.015), acc=0.8, vel=1.5)
        rob.movel_tool((0,0,0,0,0,alpha), acc=0.5, vel=0.8, wait=True)
        rob.movel_tool((0,0,0,0,beta,0), acc=0.5, vel=0.8, wait=True)
        ###########################Choose d and psi######################################
        rob.translate_tool((0.008, 0, 0), acc=0.06, vel=0.08)
        rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
        #################################################################################
        #rob.translate_tool((-0.02*sin(psi), 0, 0.02*cos(psi)), acc=0.01, vel=0.05, wait=False)
        rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        ini_wrench = wrench.wrench.torque.y
        time0 = time.time()
        num_large_force = 0
        while True:
            if num_large_force == 3:
                rob.stopl()
                break
            wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
            if wrench.wrench.torque.y>ini_wrench+0.07:   
                num_large_force += 1
            if time.time()-time0>2.8:  
                break
        time.sleep(0.5)
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        ini_wrench = wrench.wrench.torque.y
        for current_finger_length_extend in np.arange(0,40,4):
            current_finger_length = 80+current_finger_length_extend #Change
            signal.setitimer(signal.ITIMER_REAL, 0.001)
            ser.write('1'+str(current_finger_length))
            signal.setitimer(signal.ITIMER_REAL, 0)
            time.sleep(0.2)        
            current_fingerlength_int=current_finger_length
            wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
            if wrench.wrench.torque.y>ini_wrench+0.2:
                break
        print current_fingerlength_int  
        angle_step=0.2
        for angle in np.arange(2.5,-4,-angle_step):
            translate_dir_dis, extension_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
            old_fingerlength_int=current_fingerlength_int
            current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
            signal.signal(signal.SIGALRM, myHandler)
            signal.setitimer(signal.ITIMER_REAL, 0.005)
            ser.write('1'+str(current_fingerlength_int))
            signal.setitimer(signal.ITIMER_REAL, 0)   
            if abs(current_fingerlength_int-old_fingerlength_int)!=0 and current_fingerlength_int<155:
                Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
                rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
            else:
                time.sleep(0.01)
        Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
        rospy.sleep(0.5)
        rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
        #go_to_end_place()
        #rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
        #Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
        #rospy.sleep(0.5)
        Robotiq.goto(robotiq_client, pos=2*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
        #rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 
        return True
    else:
        return False

In [14]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[Go_index]+0.0105, speed=0.1, force=10, block=True)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    pos_x = rob.get_pose().pos.x
    pos_y = rob.get_pose().pos.y
    pos_z = rob.get_pose().pos.z
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x-0.005,ver_pose_Go_stone_set[Go_index][1]-pos_y+0.0015,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.8, vel=2)
    rob.translate_tool((0.005,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.7,-0.035)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.6,-0.035)), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.019, speed=0.5, force=10, block=True)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

In [15]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, -gripper_aperture_set[Go_index]/2-0.007], [0.01, -gripper_aperture_set[Go_index]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [16]:
def distane_to_bowl_center(point_coordinate):
    return sqrt((point_coordinate[0]-0.05)**2+(point_coordinate[1]-0.69)**2)

executed_time_man_array = []
executed_time_vis_array = []
stone_img_count = 0
json_error = False
if 'switching_stone_coordinate_set' in dir():
    while True:
        whether_inherent = raw_input("Whether inherent switching_stone_coordinate_set before? (y or n)")
        if whether_inherent == 'n':
            switching_stone_coordinate_set=[]
            break
        elif whether_inherent == 'y':
            break
else:
    switching_stone_coordinate_set = []
while True:
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    ver_pose_Go_stone_set = []
    gripper_direction_set = []
    gripper_aperture_set = []
    is_send_stone_img = 1
    time_count_vis_0 = time.time()
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(yaw_pitch_normal_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(gripper_aperture_set):
            break
        elif hori_Go_stone_number != None and hori_Go_stone_number == len(yaw_pitch_normal_set) and ver_Go_stone_number != None and json_error==True:
            rospy.sleep(0.5)
            break   
        rospy.sleep(0.1) 
    vis_time = time.time()-time_count_vis_0
    executed_time_vis_array.append(vis_time)
    print 'vis_time', vis_time
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    print ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    time_count_man_0 = time.time()
    priority_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            priority_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            priority_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            for index_ver in range(len(ver_pose_Go_stone_set)):
                if hori_pose_Go_stone_set[index_hori][2]<=ver_pose_Go_stone_set[index_ver][2]-0.002 and ('v', index_ver) not in priority_list:
                    priority_list.append(('v', index_ver))
                v_num = 0
                for element in priority_list:
                    if element[0]=='v':
                        v_num+=1
                if (v_num == len(ver_pose_Go_stone_set) or hori_pose_Go_stone_set[index_hori][2]>ver_pose_Go_stone_set[index_ver][2]-0.002) and ('h', index_hori) not in priority_list:
                    priority_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            if ('v', index_ver) not in priority_list:
                priority_list.append(('v', index_ver))
    print priority_list
    for index in priority_list:
        '''
        if index[0]=='v':
            if ver_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue
            if ver_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if ver_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if direct_pick_collision_check(index[1])==False:
                direct_pick_execute(index[1])
                break
        '''
        if index[0]=='h':
            if hori_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if hori_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if hori_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if distane_to_bowl_center(hori_pose_Go_stone_set[index[1]])<0.03:
            #if True:
                if skim_grasp_execute(index[1])==True:
                    man_time = time.time()-time_count_man_0
                    executed_time_man_array.append(man_time)
                    print 'man_time', man_time
                    break
    else:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x
        pos_y = rob.get_pose().pos.y
        if ver_pose_Go_stone_set == []:
            max_height = np.max(np.array(hori_pose_Go_stone_set)[:,2])
        elif hori_pose_Go_stone_set == []:
            max_height = np.max(np.array(ver_pose_Go_stone_set)[:,2])
        else:
            max_height = max(np.max(np.array(hori_pose_Go_stone_set)[:,2]), np.max(np.array(ver_pose_Go_stone_set)[:,2]))

        if max_height>0.045:
            while True:
                if_human_shuffle_finished = raw_input("Human shuffle. Finished? (y or n)")
                if if_human_shuffle_finished == 'y':
                    break
        else:
            if hori_pose_Go_stone_set==[]:
                while True:
                    if_human_shuffle_finished = raw_input("Human shuffle. Finished? (y or n)")
                    if if_human_shuffle_finished == 'y':
                        break 
            else:
                signal.signal(signal.SIGALRM, myHandler)
                signal.setitimer(signal.ITIMER_REAL, 0.005)
                ser.write('1000')
                signal.setitimer(signal.ITIMER_REAL, 0)
                if len(hori_pose_Go_stone_set)<=2:
                    sweep_index_set = range(len(hori_pose_Go_stone_set))
                else:
                    sweep_index_set = random.sample(range(len(hori_pose_Go_stone_set)), 2)
                for hori_index in sweep_index_set:
                    alpha = atan2(0.69-hori_pose_Go_stone_set[hori_index][1], 0.05-hori_pose_Go_stone_set[hori_index][0])
                    if alpha > pi/2 and alpha<=pi:
                        rotate_angle = alpha-3*pi/2
                    else:
                        rotate_angle = alpha+pi/2
                    rob.set_tcp((0.014, 0, 0.3400, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.movel_tool((0,0,0,0,0,-rotate_angle), acc=0.5, vel=0.8, wait=True) 
                    world2tcp = rob.get_pose()
                    #coordinate_x = max(hori_pose_Go_stone_set[hori_index][0], 0.008)
                    #coordinate_y = min(hori_pose_Go_stone_set[hori_index][1], 0.73)
                    rob.translate((hori_pose_Go_stone_set[hori_index][0]-world2tcp.pos.x-0.002,hori_pose_Go_stone_set[hori_index][1]-world2tcp.pos.y+0.003,0), acc=0.5, vel=0.8, wait=True)
                    rob.movel_tool((0.002,0,0,0,0,0), acc=0.1, vel=0.2, wait=True) 
                    rob.movel_tool((0,0,0,0,20*pi/180,0), acc=0.3, vel=0.8, wait=True) 
                    rob.set_tcp((0, 0, 0, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.translate((0, 0, -0.065), acc=0.5, vel=0.8, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
                    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                    ini_wrench = wrench.wrench.force.z
                    torque_memory=[]
                    while True:
                        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                        torque_memory.append(wrench.wrench.force.z)
                        if len(torque_memory)<9:
                            if wrench.wrench.force.z<ini_wrench-2:
                                rob.stopl()
                                break
                        else:
                            if wrench.wrench.force.z<max(torque_memory[0:9])-2:
                                rob.stopl()
                                break 
                    #rob.translate((0.035*cos(alpha), 0.035*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    point_to_bowl_center_length=sqrt((hori_pose_Go_stone_set[hori_index][0]-0.05)**2+(hori_pose_Go_stone_set[hori_index][1]-0.69)**2)
                    rob.translate((point_to_bowl_center_length*0.8*cos(alpha), point_to_bowl_center_length*0.8*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    #hardtip()
                    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
                    rob.movej((radians(99.10),radians(-91.67), radians(-107.60), radians(-70.72), radians(89.13), radians(-80.76)), 0.3, 0.7)

continue?1
started 0
dsag
0
vis_time 10.817471981
[[0.07166923528495489, 0.6762607272638548, 0.037], [0.04806556819213301, 0.6763330052290948, 0.037], [0.020345621556625535, 0.6998954789670391, 0.037], [0.022375272971194064, 0.6498562818689952, 0.037]]
[[-3.066732805879026, 0.29544083714371916, (-0.19630633683258517, 0.07535421449945345, -0.9732967717445085)], [-2.6981443171700548, 0.29544083714371916, (-0.04414771308210626, 0.09397455507150655, -0.9907853390615626)], [-2.483704048407583, 0.2551823906208175, (0.16511442462931306, 0.07102775926974629, -0.9710402548746064)], [-2.677945044588987, 0.29544083714371916, (0.11009706834701322, 0.10585521509076813, -0.9408060235034162)]]
[] [] []
[('h', 0), ('h', 1), ('h', 2), ('h', 3)]
Go_index ('h', 0)
[-0.9490023342836411, 0.2831840656988735, -0.1385689520002829]
((0, 1, 2, 0, -1, (0.10959288739020928, 0.6758077257198977, 0.06990688425527293), (0.1094369086846938, 0.675775338260846, 0.06999999910593033), (-0.8453083813550699, -0.175520052539

KeyboardInterrupt: 

In [17]:
((0.075-0.07166923528495489)+(0.05-0.04806556819213301)+(0.025-0.020345621556625535)+(0.025-0.022375272971194064))/4.0

0.0031360754987731264

In [18]:
((0.675-0.6762607272638548)+(0.675-0.6763330052290948)+(0.7-0.6998954789670391)+(0.65-0.6498562818689952))/4.0

-0.0005863733322459541

In [18]:
go_to_safe_place()

In [13]:
go_to_home()

In [20]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


In [19]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np
r = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)])
r.as_matrix()

In [ ]:
import numpy as np
a=np.array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
a[:,0].tolist()

In [ ]:
    current_fingerlength_int=80
    angle_step=0.2
    for angle in np.arange(3,-4,-angle_step):
        translate_dir_dis, extension_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)

In [21]:
np.mean(executed_time_man_array)

22.931058088938396

In [22]:
np.mean(executed_time_vis_array)

4.429571138487922

In [17]:
stone_img_count = 0
while True:
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_home()
    rospy.sleep(1)
    is_send_stone_img = 1

continue?1
dsag
0
continue?1
dsag
1
continue?1
dsag
2
continue?1
dsag
3
continue?1
dsag
4
continue?1
dsag
5
continue?1
dsag
6
continue?1
dsag
7
continue?1
dsag
8
continue?1
dsag
9
continue?1
dsag
10
continue?1
dsag
11
continue?1
dsag
12
continue?1
dsag
13
continue?1
dsag
14
continue?1
dsag
15
continue?1
dsag
16
continue?1
dsag
17
continue?1
dsag
18
continue?1
dsag
19
continue?1
dsag
20
continue?1
dsag
21
continue?1
dsag
22
continue?1
dsag
23
continue?1
dsag
24
continue?1
dsag
25
continue?1
dsag
26
continue?1
dsag
27
continue?1
dsag
28
continue?1
dsag
29
continue?1
dsag
30
continue?1
dsag
31
continue?1
dsag
32
continue?1
dsag
33
continue?1
dsag
34
continue?1
dsag
35
continue?1
dsag
36
continue?1
dsag
37
continue?1
dsag
38
continue?1
dsag
39
continue?1
dsag
40
continue?1
dsag
41
continue?1
dsag
42
continue?1
dsag
43
continue?1
dsag
44
continue?1
dsag
45
continue?1
dsag
46
continue?1
dsag
47
continue?1
dsag
48
continue?1
dsag
49
continue?1
dsag
50
continue?1
dsag
51
continue?1
dsag
52
con

KeyboardInterrupt: 